## **1- Importing Libraries**

In [46]:
import numpy as np 
import pandas as pd 
import json


## **2- Data Preprocessing & Feature Engineering**

In [47]:
# Load the dataset, specifying the data types for each column
youtube_data = pd.read_csv('/Users/godzilla/Desktop/Data Analysis Projects/Youtube/Project/scraped_data/youtube_data.csv')


### **2.1- Channel Data Preprocessing**

In [48]:
# Renaming columns
new_column_names = {
    'video_id': 'Video_ID',
    'channelTitle': 'Channel_Name',
    'title': 'Title',
    'description': 'Description',
    'tags': 'Tags',
    'publishedAt': 'Published_At',
    'categoryId': 'Category_ID',
    'defaultAudioLanguage': 'Audio_Language',
    'viewCount': 'View_Count',
    'likeCount': 'Like_Count',
    'dislikeCount': 'Dislike_Count',
    'commentCount': 'Comment_Count',
    'duration': 'Duration',
    'definition': 'Definition',
    'caption': 'Caption',
    'thumbnails':'Thumbnails'
}

youtube_data.rename(columns=new_column_names, inplace=True)

In [49]:
youtube_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5779 entries, 0 to 5778
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Video_ID        5779 non-null   object
 1   Channel_Name    5779 non-null   object
 2   Title           5779 non-null   object
 3   Description     5709 non-null   object
 4   Tags            5779 non-null   object
 5   Published_At    5779 non-null   object
 6   Category_ID     5779 non-null   int64 
 7   Audio_Language  5779 non-null   object
 8   Thumbnails      5779 non-null   object
 9   View_Count      5779 non-null   int64 
 10  Like_Count      5779 non-null   int64 
 11  Dislike_Count   5779 non-null   int64 
 12  Comment_Count   5779 non-null   int64 
 13  Duration        5779 non-null   object
 14  Definition      5779 non-null   object
 15  Caption         5779 non-null   bool  
dtypes: bool(1), int64(5), object(10)
memory usage: 683.0+ KB


In [50]:
youtube_data.isnull().sum()

Video_ID           0
Channel_Name       0
Title              0
Description       70
Tags               0
Published_At       0
Category_ID        0
Audio_Language     0
Thumbnails         0
View_Count         0
Like_Count         0
Dislike_Count      0
Comment_Count      0
Duration           0
Definition         0
Caption            0
dtype: int64

In [51]:
# Convert count columns to numeric columns
numeric_columns = ['View_Count', 'Like_Count', 'Comment_Count', 'Category_ID', 'Dislike_Count']
youtube_data[numeric_columns] = youtube_data[numeric_columns].apply(pd.to_numeric, errors='coerce', axis=1)


In [52]:
# convert duration to seconds
youtube_data['Duration'] = youtube_data['Duration'].apply(lambda x: pd.to_timedelta(x).total_seconds())


In [53]:
youtube_data['Audio_Language'] = youtube_data['Audio_Language'].replace({
    'en': 'English',
    'en-US': 'English',
    'ar': 'Arabic',
    'en-GB': 'English',
    'en-IN': 'English',
    'hi': 'Hindi'
})

# Create a dictionary to map channel names to audio languages
channel_audio_mapping = {
    "Ken Jee": "English",
    "Ranesh Guruparan": "English",
    "Youssef Hosni": "Arabic",
    "Sundas Khalid": "English",
    "edrea": "English",
    "Mohamed Al Assaal - اتعلم مع العسال": "Arabic",
    "Deena Gergis": "Arabic",
    "Lore So What": "English",
    "Thu Vu data analytics": "English",
    "Data With Mo": "English",
    "techTFQ": "English",
    "Learn with Lukas": "English",
    "Alex The Analyst": "English",
    "codebasics": "English",
    "Mustafa Othman": "Arabic",
    "Justin Shin": "English",
}

# Update Audio_Language column based on the dictionary mapping for null values
for channel_name, audio_language in channel_audio_mapping.items():
    condition = (youtube_data["Channel_Name"] == channel_name) & (youtube_data["Audio_Language"].isnull())
    youtube_data.loc[condition, "Audio_Language"] = audio_language

In [54]:
# Fill missing values in the 'Comment_Count' column with 0 (zero)
youtube_data["Comment_Count"].fillna(0, inplace=True)

# Fill missing values in the 'Like_Count' column with 0 (zero)
youtube_data["Like_Count"].fillna(0, inplace=True)

# Fill missing values in the 'Tags' column with the text "Not Specified"
youtube_data["Tags"].fillna("Not Specified", inplace=True)

# Fill missing values in the 'Description' column with the text "Not Specified"
youtube_data["Description"].fillna("Not Specified", inplace=True)

# Replace 'Mohamed Al Assaal - اتعلم مع العسال' with 'Mohamed Al Assaal'
youtube_data["Channel_Name"] = youtube_data["Channel_Name"].replace('Mohamed Al Assaal - اتعلم مع العسال', 'Mohamed Al Assaal')

# Replace 'hd' with 'High Definition' in the 'Definition' column
youtube_data["Definition"] = youtube_data["Definition"].replace('hd', 'High Definition')

# Replace 'sd' with 'Standard Definition' in the 'Definition' column
youtube_data["Definition"] = youtube_data["Definition"].replace('sd', 'Standard Definition')


In [55]:
# Create a new column named 'maxres_thumbnail_url' to store the extracted 'maxres' thumbnail URLs
youtube_data['maxres_thumbnail_url'] = ''

# Loop over each row in the 'thumbnails' column and extract the URL of the 'maxres' thumbnail
for index, thumbnail_data_str in enumerate(youtube_data['Thumbnails']):
    try:
        # Parse the JSON-like string representation of thumbnail data and replace single quotes with double quotes
        thumbnail_data = json.loads(thumbnail_data_str.replace("'", "\""))
        
        # Extract the 'maxres' thumbnail URL from the parsed data, or set 'URL not found' if not present
        maxres_url = thumbnail_data.get('maxres', {}).get('url', 'URL not found')
        
        # Assign the extracted URL to the corresponding cell in the 'maxres_thumbnail_url' column
        youtube_data.at[index, 'maxres_thumbnail_url'] = maxres_url
    except json.JSONDecodeError:
        # If there's an issue with JSON parsing, mark the cell with 'Invalid JSON'
        youtube_data.at[index, 'maxres_thumbnail_url'] = 'Invalid JSON'



### ***2.2- Features Engineering***

In [56]:
# Create a dictionary to map category IDs to their text categories
category_id_to_name = {
    28: "Science & Technology",
    24: "Entertainment",
    22: "People & Blogs",
    27: "Education",
    26: "Howto & Style",
    19: "Travel & Events",
    1: "Film & Animation",
    23: "Comedy",
    25: "News & Politics",
    20: "Gaming",
    17: "Sports"
}

# Replace the category IDs with their text categories
youtube_data["Category_Name"] = youtube_data["Category_ID"].replace(category_id_to_name)

In [57]:
# Convert 'Title' and 'Tags' column to strings
youtube_data['Title'] = youtube_data['Title'].astype(str)
youtube_data['Tags'] = youtube_data['Tags'].astype(str)

# Add number of tags
youtube_data['Tags_Count'] = youtube_data['Tags'].apply(lambda x: 0 if x == 'Not Specified' else len(x.split(",")))

# Title character length
youtube_data['Title_Length'] = youtube_data['Title'].apply(lambda x: len(x.split(" ")))


In [58]:
# Calculate the engagement rate
youtube_data['Engagement_rate'] = round((youtube_data['Like_Count'] + youtube_data['Comment_Count']) / youtube_data['View_Count'] * 100, 2)


In [59]:
# Define a function to categorize videos
def categorize_videos(row):
    if row["Duration"] <= 60:
        return "Short"
    elif row["Duration"] > 60 and row["Duration"] <= 3600:
        return "Regular Content" 
    else: 
        return "Podcast/Tutorials"

# Apply the categorization function to create a new Category column
youtube_data["Video_Category"] = youtube_data.apply(categorize_videos, axis=1)

In [60]:
# Get the current date as a timezone-aware datetime
current_date = pd.Timestamp.now(tz='UTC')

# Convert the 'Published_At' column to a pandas datetime
youtube_data['Published_At'] = pd.to_datetime(youtube_data['Published_At'])

# Calculate the difference between current date and 'Published_At' in days
youtube_data['Days_Since_Published'] = (current_date - youtube_data['Published_At']).dt.days

In [61]:
# Calculate the popularity score using the given formula
youtube_data['Popularity_Score'] = (
    (youtube_data['View_Count'] * 2) +
    youtube_data['Like_Count'] - youtube_data['Dislike_Count'] +
    (youtube_data['Comment_Count'] / youtube_data['Days_Since_Published'])
)

# Round the popularity_score values to 2 decimal places
youtube_data['Popularity_Score'] = youtube_data['Popularity_Score'].round(2)

In [62]:
# Calculate the shearability score using the formula
youtube_data['Shearability_Score'] = (youtube_data['View_Count'] * 2) + \
                                     youtube_data['Like_Count'] - \
                                     youtube_data['Dislike_Count'] + \
                                     (youtube_data['Comment_Count'] / youtube_data['Days_Since_Published']) * \
                                     youtube_data['Engagement_rate']

# Round the Shearability_Score values to 2 decimal places
youtube_data['Shearability_Score'] = youtube_data['Shearability_Score'].round(2)

In [63]:
# Remove trailing spaces and empty lines from the 'comment' column
youtube_data['Title'] = youtube_data['Title'].str.strip()
youtube_data["Video_ID"] = youtube_data["Video_ID"].str.strip()


In [64]:
youtube_data

,Video_ID,Channel_Name,Title,Description,Tags,Published_At,Category_ID,Audio_Language,Thumbnails,View_Count,...,Caption,maxres_thumbnail_url,Category_Name,Tags_Count,Title_Length,Engagement_rate,Video_Category,Days_Since_Published,Popularity_Score,Shearability_Score
0,l14K2EnD548,Sundas Khalid,AI Will Replace Tech Jobs: From ex-FAANG Softw...,50 job interview questions & answers 👉🏼 https:...,"['data science', 'self-taugh data scientist', ...",2023-08-07 14:10:00+00:00,28,English,{'default': {'url': 'https://i.ytimg.com/vi/l1...,4650,...,True,https://i.ytimg.com/vi/l14K2EnD548/maxresdefau...,Science & Technology,28,9,4.00,Regular Content,5,9461.20,9479.80
1,7ssLi7Ll0I0,Sundas Khalid,How Much Money I Made as Data Engineer? (3 yea...,Resume & Cover Letter template (free) 👉🏼 https...,"['data science', 'data scientist', 'self-taugh...",2023-07-28 14:10:00+00:00,28,English,{'default': {'url': 'https://i.ytimg.com/vi/7s...,16621,...,True,https://i.ytimg.com/vi/7ssLi7Ll0I0/maxresdefau...,Science & Technology,27,14,1.97,Regular Content,15,33542.00,33543.94
2,xr68cbOxvBs,Sundas Khalid,How to learn Python FAST with ChatGPT and Bard?,Try Quadratic for FREE 👉🏼 https://QuadraticHQ....,"['data science', 'data scientist', 'self-taugh...",2023-07-10 14:10:00+00:00,28,English,{'default': {'url': 'https://i.ytimg.com/vi/xr...,339321,...,True,https://i.ytimg.com/vi/xr68cbOxvBs/maxresdefau...,Science & Technology,28,9,1.22,Regular Content,33,682682.15,682682.84
3,mLP4kdk3DoI,Sundas Khalid,Will AI Replace Data Scientists?,Excel graphs template (free) 👉🏼 https://clickh...,"['data science', 'data scientist', 'self-taugh...",2023-06-29 14:12:00+00:00,28,English,{'default': {'url': 'https://i.ytimg.com/vi/mL...,18222,...,False,https://i.ytimg.com/vi/mLP4kdk3DoI/maxresdefau...,Science & Technology,27,5,3.42,Regular Content,44,36998.61,37002.52
4,znouY2A61WI,Sundas Khalid,How to code Python FAST for Data Analysis: Bar...,Click to read full AI Trend Report (FREE) 👉🏼 h...,"['data science', 'data scientist', 'self-taugh...",2023-06-16 14:10:01+00:00,28,English,{'default': {'url': 'https://i.ytimg.com/vi/zn...,23083,...,True,https://i.ytimg.com/vi/znouY2A61WI/maxresdefau...,Science & Technology,27,11,2.21,Regular Content,57,46638.68,46639.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5774,JjuCGJyZacE,Nicholas Renotte,Generating Credentials - Build An Image Classi...,Tired of struggling to build an image classifi...,"['image classification', 'python', 'ibm', 'vis...",2019-01-29 21:29:54+00:00,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Jj...,1125,...,False,https://i.ytimg.com/vi/JjuCGJyZacE/maxresdefau...,People & Blogs,5,13,1.33,Regular Content,1656,2260.00,2260.00
5775,PPq79Q51Ya4,Nicholas Renotte,Installing Watson Developer Cloud - Build An I...,Tired of struggling to build an image classifi...,"['visual recognition', 'image classification',...",2019-01-29 21:29:50+00:00,22,0,{'default': {'url': 'https://i.ytimg.com/vi/PP...,1076,...,False,https://i.ytimg.com/vi/PPq79Q51Ya4/maxresdefau...,People & Blogs,5,15,1.39,Short,1656,2161.00,2161.01
5776,oKDkwZkzX48,Nicholas Renotte,General Image Classification - Build An Image ...,Tired of struggling to build an image classifi...,"['watson', 'ibm', 'visual recognition', 'image...",2019-01-29 21:29:47+00:00,22,0,{'default': {'url': 'https://i.ytimg.com/vi/oK...,1473,...,False,https://i.ytimg.com/vi/oKDkwZkzX48/maxresdefau...,People & Blogs,5,14,1.63,Regular Content,1656,2959.01,2959.01
5777,FgZsV09npbs,Nicholas Renotte,Food Image Classification - Build An Image Cla...,Tired of struggling to build an image classifi...,"['python', 'image classification', 'watson', '...",2019-01-29 21:29:44+00:00,22,0,{'default': {'url': 'https://i.ytimg.com/vi/Fg...,2164,...,False,https://i.ytimg.com/vi/FgZsV09npbs/maxresdefau...,People & Blogs,5,14,1.39,Regular Content,1656,4341.01,4341.01


## **3- Data Cleaning**

In [65]:
# Create a list to store DataFrames to be concatenated
frames_to_concat = []

# Loop over each row and split values in the 'Tags' column
for index, row in youtube_data.iterrows():
    tags = row['Tags']
    if isinstance(tags, str):  # Ensure 'Tags' column is a string (list representation)
        # Remove square brackets and split the tags
        tags = tags.strip('[]').replace("'", "").split(', ')
        for tag in tags:
            frames_to_concat.append(pd.DataFrame({'Video_ID': [row['Video_ID']], 'Channel_Name': [row['Channel_Name']], 'Tag': [tag]}))

# Concatenate the list of DataFrames
separated_tags_data = pd.concat(frames_to_concat, ignore_index=True)

# Drop rows where 'Tag' column is equal to '0'
separated_tags_data = separated_tags_data[separated_tags_data['Tag'] != '0']




In [66]:

separated_tags_data

,Video_ID,Channel_Name,Tag
0,l14K2EnD548,Sundas Khalid,data science
1,l14K2EnD548,Sundas Khalid,self-taugh data scientist
2,l14K2EnD548,Sundas Khalid,tech jobs
3,l14K2EnD548,Sundas Khalid,big tech
4,l14K2EnD548,Sundas Khalid,Machine Learning
...,...,...,...
68552,z16aNdvgAog,Nicholas Renotte,python
68553,z16aNdvgAog,Nicholas Renotte,ibm
68554,z16aNdvgAog,Nicholas Renotte,watson
68555,z16aNdvgAog,Nicholas Renotte,image classification


In [67]:
comments_data = pd.read_excel('/Users/godzilla/Desktop/Data Analysis Projects/Youtube/Project/scraped_data/comments_data.xlsx')

In [68]:
comments_data

,video_id,comment_id,author,comment,like_count,published_at,total_replies
0,l14K2EnD548,UCteRPiisgIoHtMgqHegpWAQ,Sundas Khalid,Thanks for watching You can find the 50 interv...,1,2023-08-06T14:12:06Z,1
1,l14K2EnD548,UCOBCx96bEpL7dDdfBTFOkQQ,U Khalid,Impressed by guest she is right about AI doing...,0,2023-08-09T21:54:29Z,0
2,l14K2EnD548,UCez-OarnVeN1og0SrsrgJjA,Coder From Ukraine,NaN,0,2023-08-09T19:20:45Z,0
3,l14K2EnD548,UCYX-G0t7FYHQM_hxSdNqazw,Lorenzo Pla Serrano,I loved this video queens,0,2023-08-08T22:16:37Z,0
4,l14K2EnD548,UCodPJ-SoT1WetGEbXgIA5LA,dorcel marcs-(Je.Handicapo♿),Your girlfriend knows better than her I dont h...,0,2023-08-08T14:37:34Z,0
...,...,...,...,...,...,...,...
83181,FgZsV09npbs,UCV8GRihX4xNYf4n-MLJ_wrQ,Dhamyaa AL-Nasrawi,Good Thank you,1,2019-02-08T20:17:54Z,1
83182,z16aNdvgAog,UCQQgCa6CoK5T5LQoFtLx-Hw,Mallik Rao,Sir please do make RASA playlist Your teaching...,0,2022-10-11T20:50:41Z,0
83183,z16aNdvgAog,UC697b4kN-g4efBnV0pwP30A,backistall,Could you make a tutorial on building your own...,2,2019-02-15T12:30:38Z,1
83184,z16aNdvgAog,UC697b4kN-g4efBnV0pwP30A,backistall,Thank you very much,1,2019-02-15T12:29:42Z,1


In [69]:
names_to_exclude = ["Tina Huang", "Keith Galli", "Rob Mulla", "Seattle Data Guy", "Emma Ding", 
                    "Justin Shin", "Ken Jee", "codebasics", "Learn with Lukas", 
                    "Thu Vu data analytics", "Luke Barousse", "Nicholas Renotte", "How to Power BI",
                    "Data Nash", "Data Wizardry", "Avery Smith - Data Career Jumpstart",
                    "Python Programmer", "edrea", "Lore So What", "Darshil Parmar", 
                    "Ranesh Guruparan", "Alex The Analyst", "techTFQ", "Annamonova",
                    "Greg Hogg", "Data With Mo", "Krish Naik", "Sundas Khalid",
                    "The Almost Astrophysicist"]

# Filter out the specified author names
comments_data = comments_data.query('author not in @names_to_exclude')


In [70]:
comments_data

,video_id,comment_id,author,comment,like_count,published_at,total_replies
1,l14K2EnD548,UCOBCx96bEpL7dDdfBTFOkQQ,U Khalid,Impressed by guest she is right about AI doing...,0,2023-08-09T21:54:29Z,0
2,l14K2EnD548,UCez-OarnVeN1og0SrsrgJjA,Coder From Ukraine,NaN,0,2023-08-09T19:20:45Z,0
3,l14K2EnD548,UCYX-G0t7FYHQM_hxSdNqazw,Lorenzo Pla Serrano,I loved this video queens,0,2023-08-08T22:16:37Z,0
4,l14K2EnD548,UCodPJ-SoT1WetGEbXgIA5LA,dorcel marcs-(Je.Handicapo♿),Your girlfriend knows better than her I dont h...,0,2023-08-08T14:37:34Z,0
5,l14K2EnD548,UCvVHV5l7u25wv24pQcYgRFg,Invader G,She goes to google then does an unimaginable m...,1,2023-08-08T03:52:46Z,2
...,...,...,...,...,...,...,...
83181,FgZsV09npbs,UCV8GRihX4xNYf4n-MLJ_wrQ,Dhamyaa AL-Nasrawi,Good Thank you,1,2019-02-08T20:17:54Z,1
83182,z16aNdvgAog,UCQQgCa6CoK5T5LQoFtLx-Hw,Mallik Rao,Sir please do make RASA playlist Your teaching...,0,2022-10-11T20:50:41Z,0
83183,z16aNdvgAog,UC697b4kN-g4efBnV0pwP30A,backistall,Could you make a tutorial on building your own...,2,2019-02-15T12:30:38Z,1
83184,z16aNdvgAog,UC697b4kN-g4efBnV0pwP30A,backistall,Thank you very much,1,2019-02-15T12:29:42Z,1


In [71]:
comments_data["comment"] = comments_data["comment"].str.replace('[\[\]]', '', regex=True)
comments_data.rename(columns={'video_id': 'Video_ID'}, inplace=True)


/var/folders/ct/r49djv1j73n1sx4547rvjz1m0000gn/T/ipykernel_21498/2342055886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_data["comment"] = comments_data["comment"].str.replace('[\[\]]', '', regex=True)
/var/folders/ct/r49djv1j73n1sx4547rvjz1m0000gn/T/ipykernel_21498/2342055886.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_data.rename(columns={'video_id': 'Video_ID'}, inplace=True)


In [72]:
comments_data.isnull().sum()

Video_ID           0
comment_id        19
author            30
comment          683
like_count         0
published_at      19
total_replies      0
dtype: int64

In [73]:
comments_data.dropna(inplace=True)

/var/folders/ct/r49djv1j73n1sx4547rvjz1m0000gn/T/ipykernel_21498/2021319583.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_data.dropna(inplace=True)


## **4- Saving Data**

In [74]:
# Save the separated 'Tages' data to an Excel file
separated_tags_data.to_excel('/Users/godzilla/Desktop/Data Analysis Projects/Youtube/Project/cleaned_data/tags_data.xlsx' , index=False)

In [75]:
youtube_data.to_csv('/Users/godzilla/Desktop/Data Analysis Projects/Youtube/Project/cleaned_data/cleaned_channels_data.csv', index=False)


In [76]:
comments_data.to_excel('/Users/godzilla/Desktop/Data Analysis Projects/Youtube/Project/cleaned_data/cleaned_comments_data.xlsx', index=False)
